In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [8]:
data=pd.read_csv("Churn_Modelling.csv")

In [9]:
data.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [10]:
## preprocessing : drop irrelevant features
data= data.drop(["RowNumber","CustomerId","Surname"],axis=1)

In [11]:
## Encoding Categorical Variable
label_encoder=LabelEncoder()

In [12]:
data["Gender"]=label_encoder.fit_transform(data['Gender'])

In [13]:
#data["Gender"]=label_encoder.fit_transform(data['Gender'])

In [14]:
from sklearn.preprocessing import OneHotEncoder

In [15]:
onehot_encoder=OneHotEncoder()
data_encoder=onehot_encoder.fit_transform(data[['Geography']])

In [16]:
data_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [17]:
onehot_encoder.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [18]:
data_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [19]:
data_encoder_df= pd.DataFrame(data_encoder.toarray(),columns=onehot_encoder.get_feature_names_out(['Geography']))

In [20]:
data_encoder_df.head(5)

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [21]:
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [22]:
data=pd.concat([data.drop('Geography',axis=1),data_encoder_df],axis=1)

In [23]:
data.head(5)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [24]:
##save the encoders 
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder,file)


with open('data_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder,file)

In [25]:
y= data['Exited']
X= data.drop('Exited',axis=1)

In [26]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [27]:
scaler=StandardScaler()

In [28]:
X_train =scaler.fit_transform(X_train)
x_test=scaler.fit_transform(X_test)

In [29]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

ANN IMPLEMENTATION

In [30]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import datetime

In [31]:
(X_train.shape[1],)

(12,)

In [32]:
model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid'),
]

)

c:\ann_end2end_project\ann_venv\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [33]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [34]:
import tensorflow
opt= tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()

In [35]:
## model compile
#model.compile(optimizer="adam",loss='binary_crossentropy',metrics=['accuracy'])
model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])

In [36]:
from tensorflow.keras.callbacks import TensorBoard
##setup tensorboard
log_dir='logs/fit' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [37]:
## set up earlystopping
early_stopping_callback= EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)

In [38]:
##Train the model
history = model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8096 - loss: 0.4377 - val_accuracy: 0.6980 - val_loss: 7323.8970
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8471 - loss: 0.3635 - val_accuracy: 0.4735 - val_loss: 6979.9683
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8601 - loss: 0.3417 - val_accuracy: 0.6190 - val_loss: 4020.2166
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8567 - loss: 0.3494 - val_accuracy: 0.6980 - val_loss: 7354.9990
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8575 - loss: 0.3465 - val_accuracy: 0.5180 - val_loss: 11686.9990
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8622 - loss: 0.3324 - val_accuracy: 0.7145 - val_loss: 5315.4009
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8612 - loss: 0.3348 - val_accuracy: 0.5495 - val_loss: 8492.1689
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8665 - l

In [39]:
model.save('model.keras')

In [40]:
##load tensorboard extension
%load_ext tensorboard

In [41]:
#%tensorboard --logdir logs/fit
%tensorboard --logdir logs/fit20250725-163428

Reusing TensorBoard on port 6007 (pid 13096), started 2 days, 20:08:07 ago. (Use '!kill 13096' to kill it.)

In [42]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import pickle

In [43]:
# Load the saved model, encoder, scalar
model=load_model("model.keras")

with open("label_encoder_gender.pkl","rb") as file:
    label_encoder_loaded= pickle.load(file)

with open("data_encoder_geo.pkl","rb") as file:
    data_encoder_geo_loaded= pickle.load(file)

with open("scaler.pkl", "rb") as file:
    scaler= pickle.load(file)

In [44]:
label_encoder_loaded

LabelEncoder()

In [45]:
data_encoder_geo_loaded

,categories,'auto'
,drop,None
,sparse_output,True
,dtype,<class 'numpy.float64'>
,handle_unknown,'error'
,min_frequency,None
,max_categories,None
,feature_name_combiner,'concat'


In [46]:
data.columns

Index(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited',
       'Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype='object')

In [47]:
## Example of Input data
input_data = {
    'CreditScore': 600,
    'Geography' : 'Germany',
    'Gender':'Male',
    'Age' : 40,
    'Tenure' :3,
    'Balance': 60000,
    'NumOfProducts':2,
    'HasCrCard':1, 
    'IsActiveMember':1,
    'EstimatedSalary':50000,
}


In [48]:
geo_encoded= data_encoder_geo_loaded.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=data_encoder_geo_loaded.get_feature_names_out(['Geography']))
geo_encoded_df

c:\ann_end2end_project\ann_venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,0.0,1.0,0.0


In [49]:
input_df=pd.DataFrame([input_data])

In [50]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,Germany,Male,40,3,60000,2,1,1,50000


In [51]:
input_df['Gender'] = label_encoder_loaded.fit_transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,Germany,0,40,3,60000,2,1,1,50000


In [52]:
input_df = pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)

In [53]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,0,40,3,60000,2,1,1,50000,0.0,1.0,0.0


In [54]:
## combine onehot encoded columns with input data
#input_df = pd.concat([input_df.reset_index(drop=True),geo_encoded_df],axis=1)

In [55]:
input_scaled= scaler.transform(input_df)

In [56]:
input_scaled

array([[-0.47154541, -1.09997489,  0.09477172, -0.69844549, -0.29010416,
         0.80510537,  0.63367318,  0.95214374, -0.84805047, -1.02020406,
         1.73668197, -0.56349184]])

In [57]:
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step


array([[0.18234423]], dtype=float32)

In [58]:
prediction_prob =(prediction[0][0])

In [59]:
prediction_prob

np.float32(0.18234423)

In [60]:
if prediction_prob>0.5:
    print("The customer is likely to churn")
else:
    print("The customer is not likely to churn")

The customer is not likely to churn
